In [29]:
import datetime 
import distance
import pandas as pd
import numpy as np
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Dataset
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
from sklearn.cluster import dbscan
import sklearn.cluster
from leven import levenshtein       
from fuzzywuzzy import fuzz

In [30]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="f2ec3ef9-cca6-46ec-be61-845d74fcae94")

In [31]:
subscription_id = 'ade92c9f-9e5b-4355-812d-4d6826ffb5d0'
resource_group = 'AI-RD'
workspace_name = 'AlanML'
workspace = Workspace(subscription_id, resource_group, workspace_name)
dataset = Dataset.get_by_name(workspace, name='workplace-data')
df = dataset.to_pandas_dataframe().drop(columns = ["Column3"])


In [4]:
cases =df["RecentWorkplaces"]
datesDF = df["CreatedOnDate"]
arrayOfPlaces = []
dates = []
df["FuzzyMatch"] =""
#df.loc[[3],"FuzzyMatch"] ="test"

In [6]:
for case in cases:
    arrayOfPlaces.append(case)


In [7]:
def createMatches():
    arrayOfPlacesThatMatch = []
    Quantity = {}
    straightLength = len(arrayOfPlaces)-1
    
    for index,place in enumerate(range(straightLength)):

        Str1 = arrayOfPlaces[place]

        if Str1 not in Quantity:
            Quantity[Str1] = "" 

        for placeToMatch in range (straightLength):
            Str2 = arrayOfPlaces[placeToMatch]       
            ratio = fuzz.ratio(Str1,Str2)

            if ratio > 60:
                if Str2 not in Quantity:
                    arrayOfPlacesThatMatch.append(Str2)
                    
                    Quantity[Str1] = arrayOfPlacesThatMatch
        if arrayOfPlacesThatMatch:
            df.loc[[index],"FuzzyMatch"] = str(arrayOfPlacesThatMatch)
        arrayOfPlacesThatMatch = []
    return Quantity
Quantity=createMatches()
#df



In [8]:
dfGroupedwithFuzzyMatchColumn = df.groupby(["CreatedOnDate"])[["RecentWorkplaces","FuzzyMatch"]].sum().reset_index("CreatedOnDate")
dfGroupedwithRecentWorkplacesColumn = df.groupby(["CreatedOnDate"])[["RecentWorkplaces"]].sum().reset_index("CreatedOnDate")
both = pd.concat([dfGroupedwithFuzzyMatchColumn,dfGroupedwithRecentWorkplacesColumn], axis=0, ignore_index=True, sort=False)
#both
#dfGroupedwithRecentWorkplacesColumn
#dfGroupedwithFuzzyMatchColumn
# for i in df["RecentWorkplaces"]:
#     print(i)


In [9]:
# for i,k in dfGroupedwithRecentWorkplacesColumn.T.iteritems():
#     print(k)
# df.groupby('CreatedOnDate').agg(["FuzzyMatch"])
# for k,v in Quantity.items():
#     print("Original:", k, "--- Matches: ", v)

In [10]:
# for columnName,rowMatch in df.T.iteritems():
#     for i in rowMatch:
#         print(i)
        
        

In [12]:
# data = [dfGroupedwithFuzzyMatchColumn, dfGroupedwithRecentWorkplacesColumn]
# def lev_metric(x, y):
#     i, j = int(x[0]), int(y[0]) # extract indices
#     return levenshtein(data[i], data[j])

# X = np.arange(len(data)).reshape(-1, 1)
#dbscan(X, metric=lev_metric, eps=5, min_samples=2)

In [14]:
# X, _ = make_blobs(n_samples=10, centers=3, n_features=4)
# df = pd.DataFrame(X, columns=['Feat_1', 'Feat_2'])
# kmeans = KMeans(n_clusters=3)
# y = kmeans.fit_predict(df[['Feat_1', 'Feat_2']])
# df['Cluster'] = y
# print(df.head())

In [9]:

# data = ["ACCTCCTAGAAG", "ACCTACTAGAAGTT", "GAATATTAGGCCGA"]
# def lev_metric(x, y):
#     i, j = int(x[0]), int(y[0])     # extract indices
#     return levenshtein(data[i], data[j])

# X = np.arange(len(data)).reshape(-1, 1)
# dbscan(X, metric=lev_metric, eps=5, min_samples=2)

In [25]:
## *** Clustering Working Attempt *** 

removeable = [None,"N/A", "N/A ", "na" "n/a", "None", "na","NA"]
stringArray = []

for string in cases:
    if string not in removeable:
        stringArray.append(string)
mean_data = np.array(stringArray)

In [27]:
#test string = "Queens University", "Queens", "Queens Belfast", "Queen University Belfast".split(" ") # test
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in mean_data] for w2 in stringArray])

affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
for cluster_id in np.unique(affprop.labels_):
    exemplar = mean_data[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(mean_data[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print("\n - *%s:* %s" % (exemplar, cluster_str))



 - *works The Stables Lady Dickson Pk Belfast previously known as the Ability Café.  Reports hasn't been there for 2 months but will let manager know. No plans yet to return to work.:* works The Stables Lady Dickson Pk Belfast previously known as the Ability Café.  Reports hasn't been there for 2 months but will let manager know. No plans yet to return to work.

 - *Blair House care Home Newtownards. Agency is Staff Nursing Belfast:* Blair House care Home Newtownards. Agency is Staff Nursing Belfast

 - *St Marys and Christian Brothers Grammar, Belfast:* St Marys and Christian Brothers Grammar, Belfast

 - *Community - Health care Worker :* Community - Health care Worker , Four Seasons Healthcare Comber, Hollywood Arches, Health Centre, Belfast , Manor Healthcare Supported Living, Mental Health Recovery service, SHSCT Community Domicillary Care 

 - *Anns Home Care In Community Setting - Market Hill area :* Anns Home Care In Community Setting - Market Hill area 

 - *-Daisy Hill Hospi

In [13]:
# words = "QueensUniversity Queens QueensBelfast QueenUniversityBelfast".split(" ") #Replace this line
# words = np.asarray(words) #So that indexing with a list will work
# lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])
# print(words)

# affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=0.5)
# affprop.fit(lev_similarity)
# for cluster_id in np.unique(affprop.labels_):
#     exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
#     cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
#     cluster_str = ", ".join(cluster)
#     print(" - *%s:* %s" % (exemplar, cluster_str))

['QueensUniversity' 'Queens' 'QueensBelfast' 'QueenUniversityBelfast']
 - *Queens:* Queens, QueensBelfast
 - *QueenUniversityBelfast:* QueenUniversityBelfast, QueensUniversity
